In [1]:
# Importing the necessary libraries for our script

import json
import pandas as pd
import numpy as np
import jmespath

In [2]:
# Loading our file

file_name = "part-xxxx"
data = [json.loads(line) for line in open(file_name, 'r', encoding = 'utf-8')]

In [3]:
# Selecting only ANDROID Devices

device_list = []
for users in range(len(data)):
    device = {}
    if data[users]['device']['platform'] == 'ANDROID':
                   device_list.append(data[users])

In [4]:
# Creating a function to extract location details

device_locations = []
def extract_values(list_of_dicts, key1, key2):    
    for d in list_of_dicts:
        new_dict = {}
        if key1 in d:
            new_dict[key1] = d[key1]
        if key2 in d:
            new_dict[key2] = d[key2]
        if bool(new_dict):
            device_locations.append(new_dict)

In [5]:
# Extracting the location data along with device details

extract_values(device_list, 'location', 'device')

In [6]:
# Converting above extracted file to Dataframe in order to merge with Zip Level Housing Prices

normalised_location_level1 = pd.json_normalize(device_locations)                             # JSON to DF
normalised_location_level1['user_id'] = normalised_location_level1['device.sha1_dpid']       # Setting User_ID
location_toRows = normalised_location_level1.explode('location')                             # Exploding locations
expanded_location = location_toRows['location'].apply(pd.Series)                             # Expanding locations
# Merging with original data
location_data = pd.concat([expanded_location, location_toRows.drop(['location'], axis = 1)], axis = 1)
location_data = location_data[location_data['zip'] != '']
location_data = location_data[location_data['zip'] != 'NIL']
location_data = location_data[location_data['geohash'] != '7zzzzzzzz']                       # Dropping masked locations

In [7]:
location_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23556 entries, 2 to 1701
Data columns (total 29 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   country                 23556 non-null  object 
 1   state                   23556 non-null  object 
 2   city                    23556 non-null  object 
 3   dma                     23556 non-null  object 
 4   zip                     23556 non-null  object 
 5   geohash                 23556 non-null  object 
 6   day_part                23556 non-null  object 
 7   first_seen              23556 non-null  object 
 8   last_seen               23556 non-null  object 
 9   brq                     23556 non-null  int64  
 10  accuracy                23556 non-null  float64
 11  connection_types        23556 non-null  object 
 12  num_days                23556 non-null  int64  
 13  device.device_name      23556 non-null  object 
 14  device.device_model     23556 non-null 

In [8]:
zip_price = pd.read_excel('Zip Level Price.xlsx')                             # Reading Zillow Data
zip_price.rename(columns = {'Zip' : 'zip', 2021:'price'}, inplace = True)     # Renaming the columns
zip_price['zip'] = zip_price['zip'].astype('int64')                           # To merge datasets, datatype should match
location_data = location_data[location_data['zip'].str.len() >= 5]
location_data['zip'] = location_data['zip'].astype('int64')

In [9]:
location_with_price = pd.merge(location_data, zip_price, on = 'zip')           # Merging location with House Prices

In [10]:
location_with_price = location_with_price.dropna(subset = ['price'])                       # Dropping locations with house prices as NULL

In [11]:
# User travelled to atleast 10 unique locations

# n = int(input('How many locations? : '))
n = 5
counts = location_with_price.groupby('user_id')['zip'].nunique()
valid_user_ids = counts[counts >= n].index
atleast_n_locations = location_with_price[location_with_price['user_id'].isin(valid_user_ids)]

In [12]:
atleast_n_locations.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14073 entries, 161 to 22959
Data columns (total 30 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   country                 14073 non-null  object 
 1   state                   14073 non-null  object 
 2   city                    14073 non-null  object 
 3   dma                     14073 non-null  object 
 4   zip                     14073 non-null  int64  
 5   geohash                 14073 non-null  object 
 6   day_part                14073 non-null  object 
 7   first_seen              14073 non-null  object 
 8   last_seen               14073 non-null  object 
 9   brq                     14073 non-null  int64  
 10  accuracy                14073 non-null  float64
 11  connection_types        14073 non-null  object 
 12  num_days                14073 non-null  int64  
 13  device.device_name      14073 non-null  object 
 14  device.device_model     14073 non-nu

In [13]:
try:
    atleast_n_locations = atleast_n_locations[['user_id', 'country', 'state', 'city', 'dma', 'zip', 'county', 'geohash',
                                               'price', 'device.device_name', 'device.device_model', 'device.hwv', 'device.device_vendor',
                                               'device.device_category', 'device.manufacturer', 'device.year_released',
                                               'device.platform', 'device.major_os', 'device.md5_dpid',
                                               'device.sha1_dpid', 'device.md5_did', 'device.sha1_did', 'device.ifa',
                                               'device.user_agent']]
except KeyError:
    pass  # or you can print a message here

In [15]:
# Creating a function to extract application used by users

device_app = []
def extract_app_values(list_of_dicts, key1, key2):    
    for d in list_of_dicts:
        new_dict = {}
        if key1 in d:
            new_dict[key1] = d[key1]
        if key2 in d:
            new_dict[key2] = d[key2]
        if bool(new_dict):
            device_app.append(new_dict)

In [16]:
# Extracting application and device data

extract_app_values(device_list, 'asn_exchange', 'device')

In [17]:
# Converting the json file to dataset (csv)

normalised_app_level1 = pd.json_normalize(device_app)
normalised_app_level1['user_id'] = normalised_app_level1['device.sha1_dpid']
app_data = normalised_app_level1[normalised_app_level1['user_id'].isin(atleast_n_locations['user_id'])]
app_toRows = app_data.explode('asn_exchange')
expanded_apps = app_toRows['asn_exchange'].apply(pd.Series)
device_apps = pd.concat([expanded_apps, app_data.drop(['asn_exchange'], axis = 1)], axis = 1)
device_apps_cleaned = device_apps[device_apps['bundle'] != '']                            # Dropping blank bundles
device_apps_cleaned = device_apps_cleaned[device_apps_cleaned['bundle'] != 'NIL']
device_apps_cleaned = device_apps_cleaned[device_apps_cleaned['device.device_name'] != 'NIL']

In [18]:
# Subsetting based on minimum n apps

# n = int(input('How many apps? : '))
n = 5
counts = device_apps_cleaned.groupby('user_id')['asn'].nunique()
valid_user_ids = counts[counts >= n].index
atleast_n_apps = device_apps_cleaned[device_apps_cleaned['user_id'].isin(valid_user_ids)]

In [19]:
# Preparing device dataset

device_data = atleast_n_apps[['device.device_name', 'device.device_model',
       'device.hwv', 'device.device_vendor', 'device.device_category',
       'device.manufacturer', 'device.year_released', 'device.platform',
       'device.major_os', 'device.md5_dpid', 'device.sha1_dpid',
       'device.md5_did', 'device.sha1_did', 'device.ifa', 'device.user_agent']]
device_data_cleaned = device_data.drop_duplicates()
device_data_cleaned

,device.device_name,device.device_model,device.hwv,device.device_vendor,device.device_category,device.manufacturer,device.year_released,device.platform,device.major_os,device.md5_dpid,device.sha1_dpid,device.md5_did,device.sha1_did,device.ifa,device.user_agent
8,Q Stylus Plus,LM-Q710.FGN,,LG,NIL,LG,2018,ANDROID,ANDROID 8,8b2c1a1928ac5dc157e2b8d999f29f00,6ff63ff1d53d99e9fa19a2a909f036236da35de0,,,6cad1b71-f292-4d06-91e5-c027dbf1c4d5,Mozilla/5.0 (Linux; Android 8.1.0; LM-Q710(FGN...
19,Galaxy S7 Edge,SM-G935V,,Verizon,SMART PHONE,Samsung,2016,ANDROID,ANDROID 8,44f9dd5b392cf195516bf3126873f36b,2bcbd9a58fd24b55f2e375c239ae2e50b029284c,,,f9598e0f-3e61-47d7-a9a7-0d07ee2ffc4c,Mozilla/5.0 (Linux; Android 8.0.0; SM-G935V Bu...
24,Galaxy S9 Plus,SM-G965U,,Samsung,SMART PHONE,Samsung,2018,ANDROID,ANDROID 9,24a2994727e25a54ded5220ae16111b0,ac59e0ae2434886c9cf48c929cfcbb72444f549c,,,2636a459-bfc0-4acb-a1a9-7bd6028b47ec,Mozilla/5.0 (Linux; Android 9; SM-G965U Build/...
82,Galaxy A10e,SM-A102U,,Samsung,SMART PHONE,Samsung,2019,ANDROID,ANDROID 9,7e16fc4c8f0435ce3be5305e7e0bd985,2d8d394018fff31e57bbd939c8d144e126a3635f,,,9460b8f4-983e-416c-8f42-614636fe668d,Mozilla/5.0 (Linux; Android 9; SM-A102U Build/...
124,Galaxy S8,SM-G950U,,Samsung,SMART PHONE,Samsung,2017,ANDROID,ANDROID 9,bf26e1d212e37365e4280ed3d1be08df,c570965afbd668896df651b24fe0970b745898f0,,,5f597565-0403-4f94-bb55-01436176cc66,Mozilla/5.0 (Linux; Android 9; SM-G950U Build/...
138,Galaxy A10e,SM-A102U,,Samsung,SMART PHONE,Samsung,2019,ANDROID,ANDROID 9,c4bd4eb62d6d9e1676b9026ed7ad87ec,829579cec5216733e8a733df084ca11ca25f8938,,,3b966dfd-fbed-4c95-a939-891a872b41d0,Mozilla/5.0 (Linux; Android 9; SM-A102U Build/...
166,Aristo 3,LM-X220,,LG,SMART PHONE,LG,2019,ANDROID,ANDROID 8,7af095451066a300383bf8f6a14c350f,e032c504278e1a76b2873b06177dad83396b9499,,,8e1ec565-58e5-4dc8-b29f-15e8a0a9843d,Mozilla/5.0 (Linux; Android 8.1.0; LM-X220 Bui...
238,Galaxy S8,SM-G950U,,Samsung,SMART PHONE,Samsung,2017,ANDROID,ANDROID 9,d9f010c9cd37dcb471208cbdb40031e5,39cdb357766bc41512c02e22b439075915643a55,,,1dc949e4-31f4-4be9-80cb-be4b023831a8,Mozilla/5.0 (Linux; Android 9; SM-G950U Build/...
288,G7 ThinQ,LM-G710VM,,LG,SMART PHONE,LG,2018,ANDROID,ANDROID 9,e66486ae14951a46986406a8f452f39d,6154ebb2e98956987daf3eb28f99c7a1d60b19d4,,,44a2995e-21e1-430c-9b56-f8595fe46b0c,Mozilla/5.0 (Linux; Android 9; LM-G710VM Build...
297,ZenPad Z8s,ZT582KL (P00J),,Asus,TABLET,Asus,2017,ANDROID,ANDROID 7,8c2c3ac02955353d6430ac8311c91bb9,21a53f1a09e498c9c2f934386fb3350215871416,,,1ec5d8ce-12c6-4375-a231-a640389eac4d,Mozilla/5.0 (Linux; Android 7.0; ASUS_P00J Bui...


In [20]:
atleast_n_apps_n_locations = atleast_n_locations[atleast_n_locations['user_id'].isin(atleast_n_apps['user_id'])]

In [21]:
atleast_n_apps_n_locations['user_id'].nunique()

51

In [22]:
atleast_n_apps.to_csv(file_name + '_' + 'Atleast_5_apps.csv')
atleast_n_apps_n_locations.to_csv(file_name + '_' + 'Atleast_5_applications_5_locations.csv')
device_data_cleaned.to_csv(file_name + '_' + 'Device_data.csv')

In [43]:
all_locations_df = pd.DataFrame(columns = atleast_n_apps_n_locations.columns)

In [24]:
all_locations_df = pd.concat([atleast_n_apps_n_locations, all_locations_df], axis = 0)

In [44]:
all_locations_df.to_csv('all_locations.csv')

In [28]:
all_locations_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7779 entries, 168 to 22959
Data columns (total 30 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   country                 7779 non-null   object 
 1   state                   7779 non-null   object 
 2   city                    7779 non-null   object 
 3   dma                     7779 non-null   object 
 4   zip                     7779 non-null   object 
 5   geohash                 7779 non-null   object 
 6   day_part                7779 non-null   object 
 7   first_seen              7779 non-null   object 
 8   last_seen               7779 non-null   object 
 9   brq                     7779 non-null   object 
 10  accuracy                7779 non-null   float64
 11  connection_types        7779 non-null   object 
 12  num_days                7779 non-null   object 
 13  device.device_name      7779 non-null   object 
 14  device.device_model     7779 non-null

In [35]:
xyz = pd.read_csv('all_locations.csv')

In [42]:
all_locations_df

,country,state,city,dma,zip,geohash,day_part,first_seen,last_seen,brq,...,device.major_os,device.md5_dpid,device.sha1_dpid,device.md5_did,device.sha1_did,device.ifa,device.user_agent,user_id,price,price_category_V1
168,US,NC,Charlotte,517,28269,dnn3kj8k2,home,2020-01-07 01:54:07,2020-01-07 01:54:07,1,...,ANDROID 8,d6396123dd573ca216b08f9338891848,fad181bb74cc73569ae93f0eb3181c81f2c3d187,,,83492c75-b2c7-4a48-a87f-a6407b4a8adc,Mozilla/5.0 (Linux; Android 8.1.0; LM-X220 Bui...,fad181bb74cc73569ae93f0eb3181c81f2c3d187,283347.461016,High
203,US,MI,Detroit,505,48235,dpsc6bm3n,home,2020-01-12 22:20:21,2020-01-12 22:40:46,169,...,ANDROID 8,8b2c1a1928ac5dc157e2b8d999f29f00,6ff63ff1d53d99e9fa19a2a909f036236da35de0,,,6cad1b71-f292-4d06-91e5-c027dbf1c4d5,Mozilla/5.0 (Linux; Android 8.1.0; LM-Q710(FGN...,6ff63ff1d53d99e9fa19a2a909f036236da35de0,74161.203509,Low
204,US,MI,Detroit,505,48235,dpscxz2fv,home,2020-01-03 08:43:18,2020-01-03 08:47:34,61,...,ANDROID 8,8b2c1a1928ac5dc157e2b8d999f29f00,6ff63ff1d53d99e9fa19a2a909f036236da35de0,,,6cad1b71-f292-4d06-91e5-c027dbf1c4d5,Mozilla/5.0 (Linux; Android 8.1.0; LM-Q710(FGN...,6ff63ff1d53d99e9fa19a2a909f036236da35de0,74161.203509,Low
205,US,MI,Detroit,505,48235,dpsc6bm8b,other,2020-01-02 13:02:17,2020-01-02 13:02:17,1,...,ANDROID 8,8b2c1a1928ac5dc157e2b8d999f29f00,6ff63ff1d53d99e9fa19a2a909f036236da35de0,,,6cad1b71-f292-4d06-91e5-c027dbf1c4d5,Mozilla/5.0 (Linux; Android 8.1.0; LM-Q710(FGN...,6ff63ff1d53d99e9fa19a2a909f036236da35de0,74161.203509,Low
206,US,MI,Detroit,505,48235,dpscj2dz2,work,2020-01-08 20:26:42,2020-01-08 20:26:42,3,...,ANDROID 8,8b2c1a1928ac5dc157e2b8d999f29f00,6ff63ff1d53d99e9fa19a2a909f036236da35de0,,,6cad1b71-f292-4d06-91e5-c027dbf1c4d5,Mozilla/5.0 (Linux; Android 8.1.0; LM-Q710(FGN...,6ff63ff1d53d99e9fa19a2a909f036236da35de0,74161.203509,Low
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22322,US,NJ,Barnegat,501,8005,dr51y7cmw,other,2020-01-01 00:40:30,2020-01-01 00:40:30,1,...,ANDROID 9,118a3d52ae809caac6be550ec844a67d,27a8b90c9ed4761bc9314b88cf7f38fa75461e7e,,,dd540952-b74a-4f9c-a6d2-a95d75d430ea,Mozilla/5.0 (Linux; Android 9; SM-G965U Build/...,27a8b90c9ed4761bc9314b88cf7f38fa75461e7e,345572.074167,High
22323,US,NJ,Barnegat,501,8005,dr54r939p,home,2020-01-19 03:13:23,2020-01-19 03:38:01,5,...,ANDROID 9,118a3d52ae809caac6be550ec844a67d,27a8b90c9ed4761bc9314b88cf7f38fa75461e7e,,,dd540952-b74a-4f9c-a6d2-a95d75d430ea,Mozilla/5.0 (Linux; Android 9; SM-G965U Build/...,27a8b90c9ed4761bc9314b88cf7f38fa75461e7e,345572.074167,High
22957,US,CA,Tustin,803,92780,dn6ky7bvj,work,2020-01-10 19:25:44,2020-01-10 19:25:44,1,...,ANDROID 9,e1d949af91628eae15c5addbf48332c7,c0603582b0afb2e4c8265dac43f5d86625f4b039,,,881dbfe8-6d79-406b-8f4e-481634a6b154,Mozilla/5.0 (Linux; Android 9; SM-G960U Build/...,c0603582b0afb2e4c8265dac43f5d86625f4b039,748776.049905,High
22958,US,CA,Tustin,803,92780,9qhs6s0s9,other,2020-01-07 03:21:30,2020-01-07 03:21:30,1,...,ANDROID 9,e1d949af91628eae15c5addbf48332c7,c0603582b0afb2e4c8265dac43f5d86625f4b039,,,881dbfe8-6d79-406b-8f4e-481634a6b154,Mozilla/5.0 (Linux; Android 9; SM-G960U Build/...,c0603582b0afb2e4c8265dac43f5d86625f4b039,748776.049905,High


In [46]:
xyz.drop(columns=['Unnamed: 0'], inplace=True)

In [47]:
xyz.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7779 entries, 0 to 7778
Data columns (total 30 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   country                 7779 non-null   object 
 1   state                   7779 non-null   object 
 2   city                    7779 non-null   object 
 3   dma                     7779 non-null   int64  
 4   zip                     7779 non-null   int64  
 5   geohash                 7779 non-null   object 
 6   day_part                7779 non-null   object 
 7   first_seen              7779 non-null   object 
 8   last_seen               7779 non-null   object 
 9   brq                     7779 non-null   int64  
 10  accuracy                7779 non-null   float64
 11  connection_types        7779 non-null   object 
 12  num_days                7779 non-null   int64  
 13  device.device_name      7779 non-null   object 
 14  device.device_model     7779 non-null   

In [29]:
all_files = all_locations_df

In [30]:
# Calculate the median price for each state
state_median_price=all_files.drop_duplicates(['state', 'zip','price']).groupby(['state'])['price'].median().to_dict()

# Define the price category function
def price_category(price, state):
    if state not in state_median_price:
        return 0
    state_median = state_median_price[state]
    if price <= state_median:
        return "Low"
    else:
        return "High"

# Apply price category function to create 'price_category' column
all_files['price_category_V1'] = all_files.apply(lambda row: price_category(row['price'], row['state']), axis=1)

In [31]:
# Only keep the states with multiple zip (i.e., more than one)
temp = all_files.drop_duplicates(['state', 'zip','price'])
temp = temp[['state','city','zip','price','price_category_V1']].sort_values('state') # sort by state name

,state,city,zip,price,price_category_V1
18655,AL,Birmingham,35223,629239.237623,High
16088,AL,Birmingham,35242,429923.463464,High
16083,AL,Decatur,35603,219879.004609,Low
18666,AL,Montgomery,36117,200480.692759,Low
18665,AL,Sylacauga,35151,147628.116364,Low
...,...,...,...,...,...
18553,WA,Tacoma,98404,383156.824934,Low
18556,WA,Olympia,98516,488944.772479,Low
6867,WA,Seattle,98102,719055.338191,High
6943,WA,Seattle,98104,560462.457560,High


In [32]:
# Count number of unique zips in each state
zip_count_statewise = temp['state'].value_counts().rename_axis('state').reset_index(name = 'counts') 

# States that have atleast more than one zip (in this case atleast three)
states_tobe_retained = zip_count_statewise.loc[(zip_count_statewise['counts'] >= 3)]['state']
states_tobe_retained

# Final data of all files with filtered states
all_files = all_files.loc[all_files['state'].isin(list(states_tobe_retained))]

In [33]:
all_files.to_csv("location_with_housing_price_category.csv")

In [34]:
all_files.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7765 entries, 168 to 22959
Data columns (total 31 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   country                 7765 non-null   object 
 1   state                   7765 non-null   object 
 2   city                    7765 non-null   object 
 3   dma                     7765 non-null   object 
 4   zip                     7765 non-null   object 
 5   geohash                 7765 non-null   object 
 6   day_part                7765 non-null   object 
 7   first_seen              7765 non-null   object 
 8   last_seen               7765 non-null   object 
 9   brq                     7765 non-null   object 
 10  accuracy                7765 non-null   float64
 11  connection_types        7765 non-null   object 
 12  num_days                7765 non-null   object 
 13  device.device_name      7765 non-null   object 
 14  device.device_model     7765 non-null